In [ ]:
import requests
import pandas as pd
from datetime import datetime
import warnings
import time
import os
# Import for secure key loading
from dotenv import load_dotenv 

# Suppress warnings that might appear from pandas/requests internal operations
warnings.filterwarnings("ignore")

# =========================================================================
# ⚠️ 1. API CONFIGURATION (SECURELY LOADED)
# =========================================================================
# Load environment variables from the api_key.env file
# Ensure this file is in the same directory and contains: API_SPORTS_KEY=YOUR_KEY
load_dotenv('api_key.env') 

# IMPORTANT: Read the key from the environment variable API_SPORTS_KEY
API_KEY = os.getenv("API_SPORTS_KEY")

if not API_KEY:
    print("FATAL ERROR: API_SPORTS_KEY not found in api_key.env file or environment.")
    print("Please ensure your api_key.env file contains: API_SPORTS_KEY=YOUR_ACTUAL_KEY")
    exit(1)


# Universal Authentication Headers (Host will be set dynamically in the fetch function)
API_HEADERS = {
    'x-apisports-key': API_KEY 
}

# =========================================================================
# 2. SPORT CONFIGURATIONS (Football, Baseball, Handball - Season 2023)
# =========================================================================
SPORT_CONFIGS = [
    {
        "name": "Football",
        "base_url": "https://v3.football.api-sports.io",
        "host": "v3.football.api-sports.io",
        "endpoint": "fixtures",
        "league_id": 39,  # English Premier League (EPL)
        "season": "2023",
        "score_keys": ["fulltime", "home", "away"],
        "id_key": "fixture" 
    },
    {
        "name": "Baseball",
        "base_url": "https://v1.baseball.api-sports.io",
        "host": "v1.baseball.api-sports.io",
        "endpoint": "games",
        "league_id": 1,  # MLB
        "season": "2023",
        "score_keys": ["total", "home", "away"],
        "id_key": None 
    },
    {
        "name": "Handball",
        "base_url": "https://v1.handball.api-sports.io",
        "host": "v1.handball.api-sports.io",
        "endpoint": "games",
        "league_id": 1,  # EHF Euro (European Championship) - A common top league ID
        "season": "2023",
        "score_keys": ["total", "home", "away"],
        "id_key": None 
    }
]

# -------------------------------------------------------------------------
# --- API Helper Function ---
# -------------------------------------------------------------------------
def fetch_api_data(base_url, endpoint, host, params=None):
    """Generic function to fetch data, dynamically setting the host header."""
    url = f"{base_url}/{endpoint}"
    
    local_headers = API_HEADERS.copy()
    local_headers['x-rapidapi-host'] = host

    try:
        if not API_KEY:
            return None
            
        response = requests.get(url, headers=local_headers, params=params)
        response.raise_for_status() 
        data = response.json()
        
        if data.get('errors'):
            error_msg = data['errors']
            print(f"   ❌ API Error: {error_msg}")
            if any("plan" in str(e).lower() for e in error_msg.values()):
                 print("   💡 This often means you need to check your key/subscription for this sport.")
            return None
            
        return data['response']
        
    except requests.exceptions.RequestException as e:
        print(f"   ❌ Request failed for {url}: {e}")
        return None

# -------------------------------------------------------------------------
# --- Data Extraction Logic (Directly to CSV) ---
# -------------------------------------------------------------------------
def get_season_scores_to_csv(config):
    """
    Fetches ALL fixtures for a specified league and season and exports directly to a single CSV file.
    """
    all_games_data = []
    
    print("\n" + "="*80)
    print(f"STARTING FETCH for ALL {config['name']} fixtures (League ID: {config['league_id']}, Season: {config['season']})")
    print("="*80)

    print(f"  -> Fetching all fixtures for {config['season']} season...")
    
    api_data = fetch_api_data(
        config['base_url'], 
        config['endpoint'],
        config['host'],
        params={
            'league': config['league_id'],
            'season': config['season']
        }
    )
    
    if api_data:
        if len(api_data) > 0:
            print(f"     ✅ Retrieved {len(api_data)} total fixtures/games for the season.")
            
            for game in api_data:
                
                # 'score' for Football, 'scores' for others
                score_root = game.get('score', {}) if config['name'] == 'Football' else game.get('scores', {})

                # Extract ID and raw date/timestamp based on sport structure
                if config['id_key']: 
                    game_id = game.get(config['id_key'], {}).get('id')
                    timestamp_raw = game.get(config['id_key'], {}).get('timestamp')
                    status = game.get(config['id_key'], {}).get('status', {}).get('long')
                else: 
                    game_id = game.get('id')
                    timestamp_raw = game.get('date') 
                    status = game.get('status', {}).get('long')

                # --- DATE CONVERSION FIX ---
                game_date_str = 'N/A'
                if timestamp_raw is not None:
                    try:
                        # Attempt 1: Handle as integer/float timestamp (e.g., Football)
                        game_date_str = datetime.fromtimestamp(timestamp_raw).strftime('%Y-%m-%d %H:%M:%S')
                    except (TypeError, ValueError):
                        try:
                            # Attempt 2: Handle as ISO 8601 string (e.g., Others)
                            game_date_str = datetime.fromisoformat(timestamp_raw.replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
                        except:
                            pass 

                entry = {
                    'Game_Date': game_date_str,
                    'Sport': config['name'],
                    'League_ID': config['league_id'],
                    'Game_Status': status,
                    'Home_Team': game.get('teams', {}).get('home', {}).get('name'),
                    'Away_Team': game.get('teams', {}).get('away', {}).get('name'),
                    'Home_Score_Final': score_root.get(config['score_keys'][0], {}).get(config['score_keys'][1]),
                    'Away_Score_Final': score_root.get(config['score_keys'][0], {}).get(config['score_keys'][2]),
                    'Game_ID': game_id,
                }
                all_games_data.append(entry)
            
            # 3. Export to CSV file
            df = pd.DataFrame(all_games_data)
            output_file = f'{config["name"]}_{config["league_id"]}_SCORES_Season_{config["season"]}_FULL.csv'
            df.to_csv(output_file, index=False)
            
            print("\n" + "-"*50)
            print(f"🎉 DATA EXPORT COMPLETE for {config['name']}!")
            print(f"Total game scores processed: {len(df)}")
            print(f"File saved to: {output_file}")
            print("-" * 50)
        else:
            print(f"     ✅ Retrieved 0 fixtures for the season.")

    else:
        print(f"     ⚠️ Fetch failed or no data for the {config['season']} season.")


# =========================================================================
# EXECUTION (Loop through all defined sports)
# =========================================================================
if __name__ == "__main__":
    for config in SPORT_CONFIGS:
        get_season_scores_to_csv(config)
        # Pause briefly between fetching different sports to respect rate limits
        time.sleep(2)

✅ Football + Basketball live data saved to football_basketball_live.csv
